In [1]:
from google.colab import files
#uploaded = files.upload()

!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/data2')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile(
    {'q': "'1s9nJuoaHfKdspaEuM4GVWr7IvIIk2AcL' in parents"}).GetList()#{'q': "'1S02HappJ9ZBnkhLYg-PMMxTTd7HWsGCa' in parents"}).GetList()

for f in file_list:
  # 3. Create & download by id.
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = os.path.join(local_download_path, f['title'])
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)

title: rider1_ride4_timed10.csv, id: 1pnZ6Jj8rcunHb0ZKnoBR00Fg_kny13G5
downloading to /root/data2/rider1_ride4_timed10.csv
title: rider1_ride3_timed10.csv, id: 1izLX4U0D3ZtuajBZ0YGEKS9J8xOgEm_p
downloading to /root/data2/rider1_ride3_timed10.csv
title: rider1_ride2_timed10.csv, id: 1VRG_f8JD6EfJ_-huF7jCLr3zwZuPTEl0
downloading to /root/data2/rider1_ride2_timed10.csv
title: rider1_ride1_timed10.csv, id: 15wMuHMcG9OC8goI6MSXvpdaaC1tI7Hwg
downloading to /root/data2/rider1_ride1_timed10.csv


In [2]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import glob 
import copy
#tf.compat.v1.enable_eager_execution() 

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

#read csv -----all data------, rename the old files with _old
#all_files = glob.glob("//root//data2//*raw.csv")
#test_files = glob.glob("//root//data2//*old.csv")

#read csv rider wise, rename the test file as test_ and keep the rider files as is
all_files = glob.glob("//root//data2//rider*.csv")
test_files = glob.glob("//root//data2//test*.csv")

li=[]
for filename in all_files:
    print('importing...',filename)
    df1 = pd.read_csv(filename, index_col=None, header=0)
    print('before drop',len(df1))
    df1.replace(r"^\s*$",np.nan,inplace=True, regex=True)
    df1.replace(r"^\s*true$",1,inplace=True, regex=True)
    df1.replace(r"^\s*false$",0,inplace=True, regex=True)
    df1.dropna(axis=0,how='any',inplace=True)
    print('after drop',len(df1))
    li.append(df1)

df = pd.concat(li, axis=0, ignore_index=True)
total_trn = len(df)
print('total_train',len(df))

li1=[]
for filename in test_files:
  print('importing...',filename)
  df1=pd.read_csv(filename, index_col=None, header=0)
  #df1[['cadence','altitude_smooth','grade_smooth','velocity_smooth','watts']].str.strip()
  print('before drop1',len(df1))
  df1.replace(r"^\s*$",np.nan,inplace=True, regex=True)
  df1.replace(r"^\s*true$",1,inplace=True, regex=True)
  df1.replace(r"^\s*false$",0,inplace=True, regex=True)
  #df1.replace("",np.NaN,inplace=True)
  #print(df1['watts'].head())
  df1.dropna(axis=0,how='any',inplace=True)
  #df1.to_csv('new3182519785_dropped.csv')
  #files.download('new3182519785_dropped.csv')
  print('after drop1',len(df1))
  li1.append(df1)

test_df = pd.concat(li1, axis=0, ignore_index=True)
test_df_copy = copy.deepcopy(test_df)
#test_df=test_df.dropna()
total_tst = len(test_df)
print('total_test',total_tst)
#print(test_df['watts'].empty())
#test_df_filtered = test_df[test_df['watts'].str.strip()!=np.NaN]
print(test_df.tail())
#df1 = pd.read_csv('_5.csv')
#df=df.append(df1, ignore_index=True)

print (test_df.isnull().any())
print(test_df.columns)

importing... //root//data2/rider1_ride1_timed10.csv
before drop 1369
after drop 1369
importing... //root//data2/rider1_ride4_timed10.csv
before drop 663
after drop 663
importing... //root//data2/rider1_ride2_timed10.csv
before drop 1330
after drop 1330
total_train 3362
importing... //root//data2/test_rider1_ride3_timed10.csv
before drop1 231
after drop1 231
total_test 231
     Unnamed: 0  Unnamed: 0.1  ...  accel_sensor(m2/s)  accel_gps(m2/s)
226         226        1138.5  ...            0.004493         0.002644
227         227        1143.5  ...            0.003991         0.002848
228         228        1148.5  ...            0.005808         0.003444
229         229        1153.5  ...            0.008122         0.003439
230         230        1158.5  ...            0.007741         0.003458

[5 rows x 35 columns]
Unnamed: 0                   False
Unnamed: 0.1                 False
time                         False
timer_time                   False
distance                     F

In [3]:
#total_train 9770,8458,12962,22035
#total_test 2210,1354,2814,4707
#total_train 29315,25385,38893,66121
#total_test 6631,4064,8444,14124
#total_train 2930,,,6608
#total_test 663,,,1412
EVALUATION_INTERVAL = 5000 # How many records you wish to pass in each epoch interval goes here, range can >0 to <= training data set row count
EPOCHS = 30 #how many times do you want to iterate
TRAIN_SPLIT =  total_trn 
BUFFER_SIZE = 10000
BATCH_SIZE=32

#multivariate data

#minimal and extended feature headers. if you normalize and then use the data, remove weight from features, because it is a column of constants and will break the model.

# for all files, make sure you have the same no. of cols in old and new and in same order

header_sensor_minimal_new = ['distance','speed_from_sensor(m/s)','accel_sensor(m2/s)','grade_smooth','watts']#'weight_bike_rider_80',
header_gps_minimal_new = ['distance_calc(m)','speed_from_gps(m/s)','accel_gps(m2/s)','grade_smooth','watts']#'weight_bike_rider_80',

header_sensor_extended_new = ['distance','speed_from_sensor(m/s)','accel_sensor(m2/s)','grade_smooth','wind_resistance_sensor(N)',
                              'temp','cadence','altitude','watts']#'weight_bike_rider_80','heartrate',
header_gps_extended_new = ['distance_calc(m)','speed_from_gps(m/s)','accel_gps(m2/s)','grade_smooth','wind_resistance_gps(N)',
                           'temp','cadence','altitude','watts']#'weight_bike_rider_80','heartrate',
header_sensor_minimal_old = ['distance(m)','speed(m/s)','accel_sensor(m2/s)','grade(%)','watts' ] #,'weight_bike_rider_80'
header_gps_minimal_old = ['distance_calc(m)','speed_from_gps(m/s)','accel_gps(m2/s)','grade(%)', 'watts' ]#,'weight_bike_rider_80'

header_sensor_extended_old = ['distance(m)','speed(m/s)','accel_sensor(m2/s)','grade(%)','wind_resistance_sensor(N)',
                              'temperature(C)','cadence(rpm)','altitude(m)','watts']#'weight_bike_rider_80',
header_gps_extended_old = ['distance_calc(m)','speed_from_gps(m/s)','accel_gps(m2/s)','grade(%)','wind_resistance_gps(N)',
                           'temperature(C)','cadence(rpm)','altitude(m)','watts']#'weight_bike_rider_80',

features_considered = header_sensor_minimal_new # change header here to switch between min and extended, if riderwise then put the same header, if all files put new, then old

features = df[features_considered]

print(features.head())

test_features_considered = header_sensor_minimal_new # change header here to switch between min and extended, if riderwise then put the same header, if all files put new, then old


test_features=test_df[test_features_considered]
test_features.columns=features_considered 

print(test_features.tail())

frames = [features,test_features] 
features = pd.concat(frames)  
features[features_considered] = features[features_considered].astype(float) 

# switch on when all files - last fle of rider 4
'''
index1 = total_trn - 13306  
index2 = total_trn + 14204  # switch on when all files - first file of rider 4
train1 = features.iloc[:index1,:] # switch on when all files
test = features.iloc[index1:index2, :] # switch on when all files
train2 = features.iloc[index2:,:]# switch on when all files
frames2 = [train1,train2,test] # switch on when all files - used when all data
features = pd.concat(frames2)  # switch on when all files - used when all data

total_tst = len(test)
print(total_tst)
'''

print(features.head())
print(features.tail())
print('total dataset',len(features))


#print(features[~features.applymap(np.isreal).all(1)])
#plotting individual graphs for features
#features.plot(subplots=True)
#plt.legend(loc=1)
#plt.savefig('features_plot.png')
#files.download("features_plot.png") 

#plt.show()

     distance  speed_from_sensor(m/s)  ...  grade_smooth       watts
0   39.260000                   7.300  ... -1.800000e-01  216.700000
1   78.600000                   7.865  ...  1.110223e-17  217.750000
2  121.196667                   8.300  ... -3.333333e-02  219.833333
3  165.370000                   8.475  ... -2.500000e-03  214.550000
4  210.276000                   8.672  ... -1.400000e-01  216.480000

[5 rows x 5 columns]
         distance  speed_from_sensor(m/s)  ...  grade_smooth       watts
226  11617.122952               10.395551  ...      0.282335  265.926872
227  11670.002500               10.400614  ...      0.282325  266.105263
228  11722.939694               10.409956  ...      0.279083  266.310917
229  11775.959870               10.404993  ...      0.275304  266.241304
230  11828.983377               10.408348  ...      0.273420  266.387013

[5 rows x 5 columns]
     distance  speed_from_sensor(m/s)  ...  grade_smooth       watts
0   39.260000                   7.3

In [4]:
#TRAIN_SPLIT = len(train1)+len(train2) # used when all file
#print(TRAIN_SPLIT)

dataset = features.values.astype('float')

#data normalization

data_mean = dataset[:TRAIN_SPLIT].mean(axis=0)
data_std = dataset[:TRAIN_SPLIT].std(axis=0)
#print(data_mean)
#print(data_std)
dataset = (dataset-data_mean)/data_std

#data standardization
#min = dataset[:TRAIN_SPLIT].min()
#max = dataset[:TRAIN_SPLIT].max()
#dataset = (dataset-min)/(max-min)



#download normalized dataset
#dataset.to_csv('normalized_dataset.csv')
#files.download('normalized_dataset.csv')

#single step prediction
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False): #dataset, power, 0, 8000, 20, 0, 0, True
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index): # 20, 8000 for val : 8000 to 10000
    indices = range(i-history_size, i, step) # 20-20=0, 20, 1 = [0 to 20, with step 1], for val : 7980  to 8000 with step 1
    data.append(dataset[indices]) # 20 x 1 D array

    if single_step:
      labels.append(target[i+target_size]) # 20+0=20 target[20] val: target[8000]
    else:
      labels.append(target[i:i+target_size])

  return np.array(data), np.array(labels)

#past window of timesteps
past_history = 20
future_target = 0
STEP = 1

x_train_single, y_train_single = multivariate_data(dataset, dataset[:, -1], 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=True)

#validation set (if used) Used for TESTING currently
x_val_single, y_val_single = multivariate_data(dataset, dataset[:, -1],
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=True)

#train features mapped to corresponding powers 
train_data_single = tf.data.Dataset.from_tensor_slices((x_train_single, y_train_single))

#loading into memory
train_data_single = train_data_single.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat() #.shuffle(BUFFER_SIZE).



val_data_single = tf.data.Dataset.from_tensor_slices((x_val_single, y_val_single))
val_data_single = val_data_single.batch(BATCH_SIZE).repeat() 



single_step_model = tf.keras.models.Sequential()
single_step_model.add(tf.keras.layers.LSTM(32,
                                           input_shape=x_train_single.shape[-2:]))
single_step_model.add(tf.keras.layers.Dense(1))
#adam=tf.keras.optimizers.Adam(
#    learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
#   name='Adam')#, **kwargs)

single_step_model.compile(optimizer='Adam', loss='mae') #tf.keras.optimizers.RMSprop()

#fitting
single_step_history = single_step_model.fit(train_data_single, epochs=EPOCHS,
                                            steps_per_epoch=EVALUATION_INTERVAL)
                                            #validation_data=val_data_single,  #turn on when want to validate while training
                                            #validation_steps=10)

Epoch 1/30
5000/5000 [==============================] - 28s 6ms/step - loss: 0.0232
Epoch 2/30
5000/5000 [==============================] - 28s 6ms/step - loss: 0.0112
Epoch 3/30
5000/5000 [==============================] - 28s 6ms/step - loss: 0.0096
Epoch 4/30
5000/5000 [==============================] - 28s 6ms/step - loss: 0.0089
Epoch 5/30
5000/5000 [==============================] - 28s 6ms/step - loss: 0.0083
Epoch 6/30
5000/5000 [==============================] - 29s 6ms/step - loss: 0.0078
Epoch 7/30
5000/5000 [==============================] - 28s 6ms/step - loss: 0.0075
Epoch 8/30
5000/5000 [==============================] - 28s 6ms/step - loss: 0.0073
Epoch 9/30
5000/5000 [==============================] - 28s 6ms/step - loss: 0.0071
Epoch 10/30
5000/5000 [==============================] - 28s 6ms/step - loss: 0.0069
Epoch 11/30
5000/5000 [==============================] - 28s 6ms/step - loss: 0.0067
Epoch 12/30
5000/5000 [==============================] - 28s 6ms/step - lo

In [5]:
from sklearn import metrics
def plot_train_history(history, title):
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(loss))

  plt.figure()

  plt.plot(epochs, loss, 'b', label='Training loss')
  plt.plot(epochs, val_loss, 'r', label='Validation loss')
  plt.title(title)
  plt.legend()

  plt.show()

#only turn on if using validation
#plot_train_history(single_step_history,'Single Step Training and validation loss')

#print('Error RMSE : ', tf.sqrt(tf.reduce_mean((y_val_single - single_step_model.predict(x_val_single))**2)))


allrmse=[]
allmae=[]
c=0
yhatlist=[]
ylist=[]
for x, y in val_data_single.take((total_tst-20)//32):
  #print(y.shape)
  #print(x.shape)
  #print('Error RMSE : ', tf.sqrt(tf.reduce_mean((y[0].numpy() - single_step_model.predict(x)[0])**2)))
  #Y=round(y[0].numpy(),3)
  #Yhat=round(single_step_model.predict(x)[0][0],3)
  #YPower = (Y*data_std[-1])+data_mean[-1]
  #Yhatpower = (Yhat*data_std[-1])+data_mean[-1]
  
  #print('y: ', y[0].numpy(), YPower)
  #print('yhat: ', single_step_model.predict(x)[0], Yhatpower)
  #plot = show_plot([x[0][:, -1].numpy(), y[0].numpy(), # this can be switched on to plot
  #                 single_step_model.predict(x)[0]], 0,
  #              'Single Step Prediction',Y,Yhat)
  #plt.savefig('Lstm_'+str(c)+'.png')
  #files.download('Lstm_'+str(c)+'.png')
  c+=1
  print(c)
  #plot.show()
  rmse=[]
  mae =[]

  for j in range(0,len(y)):
    '''ylist.append(((y[j].numpy()*data_std[-1])+data_mean[-1]) if ((y[j].numpy()*data_std[-1])+data_mean[-1])>0 else 0)
    yhatlist.append(((single_step_model.predict(x)[j][0]*data_std[-1])+data_mean[-1]) if ((single_step_model.predict(x)[j][0]*data_std[-1])+data_mean[-1])>0 else 0 )
    rmse.append(tf.sqrt(tf.reduce_mean((y[j].numpy() - single_step_model.predict(x)[j])**2)))#.numpy())'''
    ylist.append((y[j].numpy()*data_std[-1])+data_mean[-1])
    #ylist.append(y[j].numpy())
    #print(y[j].numpy(),single_step_model.predict(x)[j],(y[j].numpy() - (single_step_model.predict(x)[j])))
    yhatlist.append((single_step_model.predict(x)[j][0]*data_std[-1])+data_mean[-1])
    #yhatlist.append(single_step_model.predict(x)[j][0])
    #rmse.append(tf.sqrt(tf.reduce_mean((y[j].numpy() - single_step_model.predict(x)[j][0])**2)))#.numpy())
    #rmse.append(metrics.mean_squared_error(y[j].numpy(),single_step_model.predict(x)[j][0],squared=False))
    #mae.append(metrics.mean_absolute_error(y[j].numpy(),single_step_model.predict(x)[j][0]))

  #print(rmse)
  #print(np.mean(rmse))
  #allrmse.append(np.mean(rmse))
  #allmae.append(np.mean(mae))
print('RMSE: ',metrics.mean_squared_error(ylist,yhatlist,squared=False))
print('MAE: ',metrics.mean_absolute_error(ylist,yhatlist))
print('Mean y_pred_lstm', np.mean(yhatlist))
print('MEan ytest', np.mean(ylist))
df2=pd.DataFrame({'ytest':ylist, 'yhat':yhatlist})
#df2.to_csv('pred_lstm.csv')
#print(len(all))
#single_step_model.save('rider4_timed3_header_sensor_minimal_old.h5')


1
2
3
4
5
6
RMSE:  18.517193054387814
MAE:  18.26155544105106
Mean y_pred_lstm 243.6575664216551
MEan ytest 261.91912186270616


In [ ]:
df2.to_csv('pred_lstm.csv')
files.download('pred_lstm.csv')

NameError: ignored